# Setup Code

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [2]:
train = pd.read_csv("train.csv")
test  = pd.read_csv("test.csv")

In [3]:
# drop ignored columns
drop_cols = ["id", "date", "zipcode", "Unnamed: 0"]
train = train.drop(columns=drop_cols, errors="ignore")
test  = test.drop(columns=drop_cols, errors="ignore")

# divide price
train["price"] = train["price"] / 1000
test["price"]  = test["price"] / 1000

# split X / y
y_train = train["price"].values
X_train = train.drop(columns=["price"]).values

y_test = test["price"].values
X_test = test.drop(columns=["price"]).values

In [4]:
# scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

# Problem 2

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [6]:
feature_names = train.drop(columns=["price"]).columns

coef_table = pd.DataFrame({
    "Feature": feature_names,
    "Coefficient": model.coef_
}).sort_values(by="Coefficient", key=abs, ascending=False)

coef_table

,Feature,Coefficient
8,grade,92.231475
13,lat,78.375737
11,yr_built,-67.643117
5,waterfront,63.742900
2,sqft_living,56.748837
9,sqft_above,48.290089
6,view,48.200109
15,sqft_living15,45.577658
10,sqft_basement,27.137032
1,bathrooms,18.527633


In [7]:
pred_train = model.predict(X_train)
mse_train_sklearn = mean_squared_error(y_train, pred_train)
r2_train_sklearn  = r2_score(y_train, pred_train)

pred_test = model.predict(X_test)
mse_test_sklearn = mean_squared_error(y_test, pred_test)
r2_test_sklearn  = r2_score(y_test, pred_test)

print("Train MSE:", mse_train_sklearn)
print("Train R2:", r2_train_sklearn)
print("Test MSE:", mse_test_sklearn)
print("Test R2:", r2_test_sklearn)

Train MSE: 31486.16777579488
Train R2: 0.7265334318706018
Test MSE: 57628.154705670386
Test R2: 0.6543560876120954


# Problem 3

In [8]:
# this function is used in problems 3,4,5,& 6
def add_bias(X):
    return np.c_[np.ones(X.shape[0]), X]

In [9]:
Xb_train = add_bias(X_train)
Xb_test  = add_bias(X_test)

In [10]:
theta, *_ = np.linalg.lstsq(Xb_train, y_train, rcond=None)

In [11]:
intercept_cf = theta[0]
coef_cf = theta[1:]

print("Intercept:", intercept_cf)

Intercept: 520.4148340000002


In [12]:
pred_train = Xb_train @ theta
pred_test  = Xb_test @ theta

In [13]:
mse_train = mean_squared_error(y_train, pred_train)
r2_train  = r2_score(y_train, pred_train)

mse_test = mean_squared_error(y_test, pred_test)
r2_test  = r2_score(y_test, pred_test)

print("Train MSE:", mse_train)
print("Train R2:", r2_train)
print("Test MSE:", mse_test)
print("Test R2:", r2_test)

Train MSE: 31486.16777579488
Train R2: 0.7265334318706018
Test MSE: 57628.1547056704
Test R2: 0.6543560876120953


In [14]:
comparison = pd.DataFrame({
    "Metric": [
        "Intercept",
        "Train MSE",
        "Train R2",
        "Test MSE",
        "Test R2"
    ],
    "Sklearn (Problem 2)": [
        model.intercept_,
        mse_train_sklearn,
        r2_train_sklearn,
        mse_test_sklearn,
        r2_test_sklearn
    ],
    "Closed Form (Problem 3)": [
        intercept_cf,
        mse_train,
        r2_train,
        mse_test,
        r2_test
    ]
})
comparison

,Metric,Sklearn (Problem 2),Closed Form (Problem 3)
0,Intercept,520.414834,520.414834
1,Train MSE,31486.167776,31486.167776
2,Train R2,0.726533,0.726533
3,Test MSE,57628.154706,57628.154706
4,Test R2,0.654356,0.654356


# Problem 4

In [15]:
feature_names = train.drop(columns=["price"]).columns

sqft_idx = list(feature_names).index("sqft_living")

x_train = X_train[:, sqft_idx]
x_test  = X_test[:, sqft_idx]

In [16]:
def poly_features(x, p):
    return np.column_stack([x**k for k in range(1, p+1)])

def fit_closed_form(X, y):
    Xb = add_bias(X)
    theta = np.linalg.pinv(Xb.T @ Xb) @ (Xb.T @ y)
    return theta

def predict_closed_form(X, theta):
    Xb = add_bias(X)
    return Xb @ theta

In [17]:
rows = []

for p in [1, 2, 3, 4, 5]:   # p ≤ 5
    Xp_train = poly_features(x_train, p)
    Xp_test  = poly_features(x_test, p)

    theta_p = fit_closed_form(Xp_train, y_train)

    pred_train = predict_closed_form(Xp_train, theta_p)
    pred_test  = predict_closed_form(Xp_test, theta_p)

    rows.append({
        "p": p,
        "Train MSE": mean_squared_error(y_train, pred_train),
        "Train R2":  r2_score(y_train, pred_train),
        "Test MSE":  mean_squared_error(y_test, pred_test),
        "Test R2":   r2_score(y_test, pred_test),
    })

results = pd.DataFrame(rows)
results

,p,Train MSE,Train R2,Test MSE,Test R2
0,1,57947.526161,0.496709,88575.978543,0.468736
1,2,54822.665116,0.523849,71791.679479,0.569406
2,3,53785.194716,0.532860,99833.483763,0.401216
3,4,52795.774758,0.541453,250979.274285,-0.505331
4,5,52626.111955,0.542927,570616.914820,-2.422464


# Problem 5

In [18]:
Xb_train = add_bias(X_train)
Xb_test  = add_bias(X_test)

In [19]:
def gradient_descent(X, y, alpha, iters, divergence_thresh=1e12):
    X = np.asarray(X, dtype=np.float64)
    y = np.asarray(y, dtype=np.float64)

    n, d = X.shape
    theta = np.zeros(d, dtype=np.float64)

    prev_loss = np.inf
    for t in range(iters):
        y_hat = X @ theta
        err = y_hat - y
        grad = (2.0 / n) * (X.T @ err)
        theta = theta - alpha * grad

        # divergence / overflow checks
        if not np.all(np.isfinite(theta)):
            return theta, "diverged"

        # stop if loss becomes absurdly large
        loss = float(np.mean(err**2))
        if loss > divergence_thresh:
            return theta, "diverged"

        # stop if loss isn't improving and starts blowing up
        if loss > prev_loss * 10 and t > 2:
            return theta, "diverged"

        prev_loss = loss

    return theta, "ok"

In [20]:
alphas = [0.01, 0.1, 0.5]
iters_list = [10, 50, 100]

rows = []
for alpha in alphas:
    for it in iters_list:
        theta, status = gradient_descent(Xb_train, y_train, alpha, it)

        if status != "ok":
            rows.append({
                "alpha": alpha, "iters": it,
                "train_mse": np.nan, "train_r2": np.nan,
                "test_mse": np.nan,  "test_r2": np.nan,
                "theta0(intercept)": np.nan,
                "theta_norm": np.nan,
                "status": status
            })
            continue

        pred_train = Xb_train @ theta
        pred_test  = Xb_test  @ theta

        # prediction safety
        if (not np.all(np.isfinite(pred_train))) or (not np.all(np.isfinite(pred_test))):
            rows.append({
                "alpha": alpha, "iters": it,
                "train_mse": np.nan, "train_r2": np.nan,
                "test_mse": np.nan,  "test_r2": np.nan,
                "theta0(intercept)": np.nan,
                "theta_norm": np.nan,
                "status": "diverged"
            })
            continue

        rows.append({
            "alpha": alpha, "iters": it,
            "train_mse": mean_squared_error(y_train, pred_train),
            "train_r2": r2_score(y_train, pred_train),
            "test_mse": mean_squared_error(y_test, pred_test),
            "test_r2": r2_score(y_test, pred_test),
            "theta0(intercept)": theta[0],
            "theta_norm": np.linalg.norm(theta),
            "status": status
        })

results_gd = pd.DataFrame(rows).sort_values(["alpha", "iters"]).reset_index(drop=True)
results_gd

,alpha,iters,train_mse,train_r2,test_mse,test_r2,theta0(intercept),theta_norm,status
0,0.01,10,235727.769775,-1.047365,280568.705498,-0.682804,95.198025,122.188353,ok
1,0.01,50,69720.498897,0.394457,97049.540753,0.417913,330.895530,363.373521,ok
2,0.01,100,36820.349872,0.680205,63333.035122,0.620139,451.397650,482.270781,ok
3,0.10,10,35105.101932,0.695102,61630.433497,0.630351,464.535717,495.278259,ok
4,0.10,50,31497.261240,0.726437,57722.475260,0.653790,520.407406,552.579981,ok
5,0.10,100,31486.431792,0.726531,57638.957225,0.654291,520.414834,553.180917,ok
6,0.50,10,NaN,NaN,NaN,NaN,NaN,NaN,diverged
7,0.50,50,NaN,NaN,NaN,NaN,NaN,NaN,diverged
8,0.50,100,NaN,NaN,NaN,NaN,NaN,NaN,diverged


# Problem 6

In [21]:
Xb_train = add_bias(X_train)
Xb_test  = add_bias(X_test)

In [22]:
def ridge_gradient_descent(X, y, alpha, iters, lam, divergence_thresh=1e12):
    X = np.asarray(X, dtype=np.float64)
    y = np.asarray(y, dtype=np.float64)

    n, d = X.shape
    theta = np.zeros(d, dtype=np.float64)

    prev_loss = np.inf
    for t in range(iters):
        y_hat = X @ theta
        err = y_hat - y

        grad = (2.0 / n) * (X.T @ err)

        # L2 penalty (exclude intercept)
        reg = np.zeros_like(theta)
        reg[1:] = 2.0 * lam * theta[1:]
        grad = grad + reg

        theta = theta - alpha * grad

        # divergence checks
        if not np.all(np.isfinite(theta)):
            return theta, "diverged"

        loss = float(np.mean(err**2) + lam * np.sum(theta[1:]**2))
        if loss > divergence_thresh:
            return theta, "diverged"
        if loss > prev_loss * 10 and t > 2:
            return theta, "diverged"
        prev_loss = loss

    return theta, "ok"

In [23]:
# run a small grid of lambdas
# 0.0 = "no regularization" baseline
lambdas = [0.0, 0.01, 0.1, 1.0, 10.0]
alpha = 0.1
iters = 200

rows = []
for lam in lambdas:
    theta, status = ridge_gradient_descent(Xb_train, y_train, alpha, iters, lam)

    if status != "ok":
        rows.append({
            "lambda": lam,
            "alpha": alpha,
            "iters": iters,
            "train_mse": np.nan,
            "train_r2": np.nan,
            "test_mse": np.nan,
            "test_r2": np.nan,
            "theta0(intercept)": np.nan,
            "theta_norm": np.nan,
            "theta_norm_no_bias": np.nan,
            "status": status
        })
        continue

    pred_train = Xb_train @ theta
    pred_test  = Xb_test  @ theta

    rows.append({
        "lambda": lam,
        "alpha": alpha,
        "iters": iters,
        "train_mse": mean_squared_error(y_train, pred_train),
        "train_r2": r2_score(y_train, pred_train),
        "test_mse": mean_squared_error(y_test, pred_test),
        "test_r2": r2_score(y_test, pred_test),
        "theta0(intercept)": theta[0],
        "theta_norm": np.linalg.norm(theta),
        "theta_norm_no_bias": np.linalg.norm(theta[1:]),
        "status": status
    })

results_ridge = pd.DataFrame(rows).sort_values("lambda").reset_index(drop=True)
results_ridge

,lambda,alpha,iters,train_mse,train_r2,test_mse,test_r2,theta0(intercept),theta_norm,theta_norm_no_bias,status
0,0.00,0.1,200,31486.168042,0.726533,57628.527406,0.654354,520.414834,553.284664,187.862502,ok
1,0.01,0.1,200,31489.595578,0.726504,57717.546927,0.653820,520.414834,552.660825,186.017172,ok
2,0.10,0.1,200,31725.388219,0.724456,58651.190005,0.648220,520.414834,548.488534,173.228382,ok
3,1.00,0.1,200,38485.180264,0.665745,69590.247493,0.582609,520.414834,534.204209,120.592444,ok
4,10.00,0.1,200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,diverged
